# Temel Bileşen Analizi

Bu ısınma alıştırması, **Temel Bileşen Analizi**'nin nasıl çalıştığına dair sezgisel bir fikir edinmenize yardımcı olacaktır. 

## (1) Veri Oluşturma

**100 gözlem** ve **birbiriyle ilişkili (korelasyonlu) 2 özellik** içeren bir veri seti istiyoruz.

👇 Verinizi oluşturmak için aşağıdaki hücreyi çalıştırın  
💡 İki özellik arasındaki (yüksek) korelasyona dikkat edin.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 100 gözlem ve 2 ilişkili özellik içeren bir veri kümesi oluşturun.
seed = np.random.RandomState(42)
feature_1 = seed.normal(5, 1, 100)
feature_2 = .7 * feature_1 + seed.normal(0, .5, 100)
X = np.array([feature_1, feature_2]).T
X = pd.DataFrame(X)

X.corr().round(3)

❓ İki özelliğinizi birbirine göre dağılım grafiği çizin.

In [ ]:
# SENİN KODUN BURAYA

☝️ Özellikler arasındaki **pozitif korelasyonu** tespit edebilirsiniz.

Gözlemlerimiz tek bir çizgi boyunca sıkışmış durumda; aralarındaki farkları ayırt etmek kolay değil.

💡 PCA, bu korelasyonu ortadan kaldıran yönleri (diğer adıyla **“principal components” – temel bileşenler**) bulmamıza yardımcı olacaktır.

## (2) Temel Bileşenler

👉 `sklearn`'den `PCA`'yı içe aktarın ve `n_components=2` ile bir model oluşturun

❓Bunu `X`'inize uyarlayın ve `pca`'ya atayın

In [ ]:
# SENİN KODUN BURAYA

Eğitilmiş `PCA` nesnesinin üç özniteliğine odaklanalım:

* `pca.components_`: Varyansın en iyi şekilde açıklandığı yönleri gösteren özvektörler kümesi; yani **maksimum varyans yönleri** (konvansiyon gereği birim uzunluğa normalize edilmişlerdir).

* `pca.explained_variance_`:  
  $Var(\text{Temel Bileşen})$, her bir temel bileşenin açıkladığı varyans (kovaryans matrisi  
  $ \Sigma = \frac{1}{n - 1} X^T X $’in özdeğerleri ile verilir).

* `pca.explained_variance_ratio_`:  
  $ \frac{Var(\text{Temel Bileşen})}{\text{Toplam Varyans}}$, her bir temel bileşenin toplam varyansın ne kadarını açıkladığını gösterir.

In [ ]:
# Her bir ana bileşenin vektör koordinatları
pca.components_

In [ ]:
# Her bir ana bileşen için açıklanan varyans
# Önem sırasına göre azalan şekilde sıralanmıştır
pca.explained_variance_

In [ ]:
# Her bir ana bileşen için açıklanan varyans oranı
# Önem sırasına göre azalan şekilde sıralanmıştır

# Her bir ana bileşen için açıklanan varyans
# verilerin toplam varyansına bölünmüştür
pca.explained_variance_ratio_

👇 Aşağıdaki hücreyi çalıştırarak iki ana bileşeninizi görselleştirin.

In [ ]:
plt.figure(figsize=(5,5))

plt.scatter(X[0], X[1])

for (length, vector) in zip(pca.explained_variance_, pca.components_):
    v = vector * np.sqrt(length) # Aynı “birimleri” karşılaştırmak için uzunluklarının karekökü
    plt.quiver(*X.mean(axis=0), *v, units='xy', scale=1, color='r')

💡 Vektörün uzunluğu, veri kümesinin bu eksene yansıtıldığında standart sapmasının bir ölçüsüdür!

💡 Daha sonra bu yönleri (bu “ana bileşenleri”) kullanarak gözlemlerimizin davranışlarının çoğunu “açıklayabiliriz” - gözlemler arasındaki farkların çoğu bu eksenler boyunca gerçekleşir. 

## (3) PCA uygulayın

Bu bileşenleri kullanarak veri setimizdeki her bir örneği, maksimum varyans yönlerine projekte edebiliriz.

❓ ***Sorular*** ❓

1. `pca` nesnesinin `transform` metodunu `X` üzerinde kullanın ve sonucu `X_transformed` değişkenine kaydedin.  
2. `X_transformed` içindeki projekte edilmiş özellikleri birbirlerine karşı grafiğe dökün.  
3. `X_transformed` içindeki dönüştürülmüş özellikler arasındaki korelasyonu hesaplayın.

In [ ]:
# SENİN KODUN BURAYA

In [ ]:
# SENİN KODUN BURAYA

In [ ]:
# SENİN KODUN BURAYA

☝️ Dönüştürülmüş özellikleriniz arasında hiçbir korelasyon yoktur.  

Bu, gözlemler arasında davranışları incelemek daha kolay hale getirir, çünkü artık tek bir çizgi üzerinde toplanmış değildirler.

🏁 **Not defterinizi PUSH etmeyi unutmayın.**  

Günün görevlerini yerine getirin ve zamanınız olursa buraya geri dönün 😉

## (Opsiyonel) Scaling'in küçük bir yardımıyla

`PCA` ile elde edilen **projeksiyonların**, aslında başlangıçtaki `X` matrisi ile *transpoze edilmiş* bileşenlerin (yani `pca.components_` ile elde edilen özvektörlerin) noktasal çarpımından ibaret olduğunu unutmayın.

👉 Noktasal çarpımı kullanarak projekte edilmiş değerleri **manuel olarak** hesaplayın:  
$X \cdot PC^T$

❓ `np.allclose` kullanarak, elde ettiğiniz `X_transformed` ile hesapladığınız  $X \cdot PC^T$ sonucunun **aynı olup olmadığını** kontrol edin.

In [ ]:
# SENİN KODUN BURAYA

Eşit değil 😱  

`sklearn`'in `PCA`'sı indirgemeyi uyguladığında, bunu *merkezlenmiş* ancak *ölçeklenmemiş* bir `X` üzerinde yapar.

Bu, `PCA().transform(X)`'in aslında `np.dot(X - X.mean(axis=0), PC)` ile eşdeğer olduğu anlamına gelir.

In [ ]:
np.allclose(X_transformed, np.dot(X - X.mean(axis=0), PC))

Bu nedenle, `PCA` kullanırken, verilerinizi öncelikle ölçeklendirmenizi, daha doğrusu standartlaştırmanızı ve böylece verilerin merkezlenmesini öneririz.

Bu şekilde, `PCA.transform()` kullanmak veya ana bileşenlerle nokta çarpımı yapmak eşdeğer olacaktır.